In [19]:
import pandas as pd
import numpy as np

In [73]:
# info, reviews를 하나로 통합

# all info 모으기
naver = pd.read_csv("naver_spot_info.csv")
kakao = pd.read_csv("kakao_spot_info.csv")
google = pd.read_csv("google_spot_info.csv")

# all reviews 모으기
# naver = pd.read_csv("naver_store_reviews.csv")
# kakao = pd.read_csv("kakao_store_reviews.csv")
# google = pd.read_csv("google_store_reviews.csv")

In [74]:
print(naver.shape)
print(kakao.shape)
print(google.shape)


(350, 9)
(377, 8)
(377, 8)


In [75]:
# main_category 빈 컬럼 추가
kakao.insert(kakao.columns.get_loc('category'), 'main_category', np.nan)
google.insert(google.columns.get_loc('category'), 'main_category', np.nan)

In [78]:
# kakao 데이터에서 naver의 id에 해당하는 행만 필터링
filtered_kakao = kakao[kakao['id'].isin(naver['id'])]
filtered_google = google[google['id'].isin(naver['id'])]
# 결과 확인
print(filtered_kakao.shape)
print(filtered_google.shape)

(350, 9)
(350, 9)


In [80]:
# 데이터 통합
merged = pd.concat([naver, filtered_kakao, filtered_google], ignore_index=True)

# 'domain' 정렬 우선순위 설정
domain_order = {"naver": 1, "kakao": 2, "google": 3}
merged["domain_order"] = merged["domain"].map(domain_order)

# id와 domain_order 기준으로 정렬
merged = merged.sort_values(by=["id", "domain_order"]).drop(columns=["domain_order"]).reset_index(drop=True)

# 결과 확인
print(merged)

       id  domain       name main_category  category  rating  \
0       1   naver        인왕산          자연명소         산     NaN   
1       1   kakao        인왕산           NaN         산     4.8   
2       1  google        인왕산           NaN    자연보호구역     4.7   
3       2   naver        북악산          자연명소         산     NaN   
4       2   kakao        북악산           NaN         산     4.3   
...   ...     ...        ...           ...       ...     ...   
1045  376   kakao   예스24스테이지           NaN  공연장,연극극장     3.6   
1046  376  google   예스24스테이지           NaN   공연예술 극장     4.3   
1047  377   naver  예스24아트원2관          공연전시      부속시설     NaN   
1048  377   kakao        NaN           NaN       NaN     NaN   
1049  377  google   예스24 아트원           NaN       NaN     5.0   

                  address                                     business_hours  \
0         서울 종로구 무악동 산2-1                                                NaN   
1       서울 종로구 옥인동 산 3-39                                              

In [67]:
# 네이버의 name 값이 없는 id 리스트
id_with_no_name = merged[merged['domain'] == 'naver'][merged['name'].isna()]['id'].unique()
print(len(id_with_no_name))

0


/var/folders/dd/p03cpd850nx3bbxblv922vsm0000gn/T/ipykernel_99796/2218451105.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  id_with_no_name = merged[merged['domain'] == 'naver'][merged['name'].isna()]['id'].unique()


In [47]:
# 네이버의 name이 없는 id에 대해 카카오와 구글 데이터의 나머지 컬럼을 NaN으로 변경
# for _id in id_with_no_name:
#     merged.loc[(merged['id'] == _id) & (merged['domain'] != 'naver'), ['name', 'main_category', 'category', 'rating', 'address', 'business_hours', 'price_per_one']] = pd.NA

# 네이버의 name이 없는 id에 대해 카카오와 구글 데이터의 행을 삭제
for _id in id_with_no_name:
    merged = merged.drop(merged[(merged['id'] == _id)].index)

# 결과 확인
print(merged)

         id  domain                     name main_category    category  \
24        9   naver                    서울메이트            주점      바(BAR)   
25        9   kakao                      NaN           NaN         NaN   
26        9  google  서울메이트 SEOUL MATE ソウルメイト           NaN          술집   
30       11   naver                     막회주자           음식점         생선회   
31       11   kakao                      NaN           NaN         NaN   
...     ...     ...                      ...           ...         ...   
19801  6601   kakao                      NaN           NaN         NaN   
19802  6601  google                     고흥식당           NaN         음식점   
19812  6605   naver                 콘크리트 팔레트            카페      카페,디저트   
19813  6605   kakao                      NaN           NaN         NaN   
19814  6605  google         Concrete Palette           NaN  커피숍/커피 전문점   

       rating                  address  \
24        NaN         서울 종로구 종로 298 2층   
25        NaN              

In [81]:
# name 컬럼이 NaN인 행을 삭제
merged = merged.dropna(subset=['name'])

# 결과 확인
print(merged)

       id  domain       name main_category  category  rating  \
0       1   naver        인왕산          자연명소         산     NaN   
1       1   kakao        인왕산           NaN         산    4.80   
2       1  google        인왕산           NaN    자연보호구역    4.70   
3       2   naver        북악산          자연명소         산     NaN   
4       2   kakao        북악산           NaN         산    4.30   
...   ...     ...        ...           ...       ...     ...   
1044  376   naver   예스24스테이지          공연전시   공연,연극시설    4.48   
1045  376   kakao   예스24스테이지           NaN  공연장,연극극장    3.60   
1046  376  google   예스24스테이지           NaN   공연예술 극장    4.30   
1047  377   naver  예스24아트원2관          공연전시      부속시설     NaN   
1049  377  google   예스24 아트원           NaN       NaN    5.00   

                  address                                     business_hours  \
0         서울 종로구 무악동 산2-1                                                NaN   
1       서울 종로구 옥인동 산 3-39                                              

In [82]:
# csv 저장
merged.to_csv("all_spot_info_2.csv", index=False)
# merged.to_csv("all_store_info_2.csv", index=False)


In [90]:
# 리뷰~
info = pd.read_csv("all_spot_info_2.csv")
reviews = pd.read_csv("all_spot_reviews.csv")
print(info.shape)
print(reviews.shape)

(867, 9)
(13749, 4)


In [91]:
# info의 id를 기준으로 reviews에서 해당하는 행만 필터링
filtered_reviews = reviews[reviews['id'].isin(info['id'])]

# 결과 확인
print(filtered_reviews)

        id  domain       name  \
0        1   naver        인왕산   
1        1   naver        인왕산   
2        1   naver        인왕산   
3        1   naver        인왕산   
4        1   naver        인왕산   
...    ...     ...        ...   
13744  376  google   예스24스테이지   
13745  377   naver  예스24아트원2관   
13746  377   naver  예스24아트원2관   
13747  377   naver  예스24아트원2관   
13748  377  google   예스24 아트원   

                                                 reviews  
0      블로그 소일소행 포스팅 참고하셔요!\nhttps://m.blog.naver.com/...  
1      노을지는 시간대에 방문하면 딱이에요! 경복궁역에서 걸어가는 길이 가장 편합니다. 초...  
2      겨울철 눈이 오는 날이면 바로 준비하고 인왕산으로 향한다! 이유는 미친 뷰에 눈꽃산...  
3      블로그를 보면서 갔지만 생각보다 어려웠던 길!\n무악재역쪽 길로 출발해서 단군성전쪽...  
4      독립문역에서 올라가는 인왕산길 눈이 와서\n그 경치가 뭐라 말할 수 없이 아름답네요...  
...                                                  ...  
13744                    초행길 찾아가느라 애먹었어요. 그닥 길치도 아닌데 ㅠㅠ.  
13745                              최재웅 배우님의 나레이터는 최고입니다!  
13746  놐놐놐.\n미드나잇앤틀러스 봤어요.\n연극은 강추하는데\n공연장 자체는 그렇게 좋은...  
13747        

In [93]:
print(len(filtered_reviews['id'].unique()))
print(len(naver['id'].unique()))

344
350


In [88]:
# naver의 id와 filtered_reviews의 id를 각각 집합으로 변환
naver_ids = set(naver['id'].unique())
reviews_ids = set(filtered_reviews['id'].unique())

# naver에만 있는 id
naver_only_ids = naver_ids - reviews_ids
print(f"naver에만 있는 id: {naver_only_ids}")

# filtered_reviews에만 있는 id
reviews_only_ids = reviews_ids - naver_ids
print(f"filtered_reviews에만 있는 id: {reviews_only_ids}")

naver에만 있는 id: {np.int64(96), np.int64(357), np.int64(168), np.int64(363), np.int64(179), np.int64(342)}
filtered_reviews에만 있는 id: set()


In [94]:
# csv 저장
# merged.to_csv("all_spot_reviews_2.csv", index=False)
filtered_reviews.to_csv("all_spot_reviews_2.csv", index=False)